In [11]:
import sys
import pandas as pd
import numpy as np
import math
data=pd.read_csv('/home/zjy/ml hw1/train.csv',encoding = 'big5') #用pandas读入csv文件

In [12]:
data = data.iloc[:,3:] #将第四列及以后的表格裁剪下来
data[data=='NR']=0 #只能识别数字，将是否降雨用0和1表示
num_data=data.to_numpy() #将csv文件data转换成numpy矩阵num_data

In [13]:
#为了使后续每9h取样时数据尽可能多且方便取样，将竖着的原始表格变换成了横着的row_data，使小时数全部排在同一行
row_data={} #先建一个空字典
for month in range(12):
    monthblock=np.empty([18,480])
    for day in range(20):
        monthblock[:,(day*24):((day+1))*24]=num_data[18*(20*month+day):18*(20*month+day+1),:] #从day到month，从里到外进行变换
    row_data[month]=monthblock #month键的值monthblock

In [14]:
#将row_data每10h分组，每9h放在x里（data），剩余1h放在y里（label）,tips:此处均采用滑动窗口进行分组
x=np.empty([12*471,18*9],dtype=float)
y=np.empty([12*471,1],dtype=float)
for month in range(12):
    for day in range(20):
        for hours in range(24):
            if day==19 and hours>14: #??
                continue
            x[month*471+day*24+hours,:]=row_data[month][:,day*24+hours:day*24+hours+9].reshape(1,-1) #reshape将x压缩为一行
            y[month*471+day*24+hours,0]=row_data[month][9,day*24+hours+9]

In [15]:
#进行数据标准化（z标准化）
mean_x=np.mean(x,axis=0) #定义求平均值
std_x=np.std(x,axis=0) #定义求标准差
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j]!=0:
            x[i][j]=(x[i][j]-mean_x[j]) / std_x[j]

In [16]:
#训练模型 loss function和gradient decent 采用Root Mean Square Error
dim=18*9+1 #dim用来作参数维数，一共18*9个w和一个b
w=np.zeros([dim,1])
x=np.concatenate((np.ones([12*471,1]),x),axis=1).astype(float)
#pretesting,对数据进按8：2进行拆分，2成为validation set,用于提前验证private set上结果好坏
x_train=x[:math.floor(len(x)*0.8),:] #math.floor 向下取整
y_train=y[:math.floor(len(y)*0.8),:]
x_validation=x[math.floor(len(x)*0.8):,:]
y_validation=x[math.floor(len(y)*0.8):,:]
learning_rate=100
iter_time=1000
adagrad=np.zeros([dim,1])
eps=0.0000000001
for t in range(iter_time):
    loss=np.sqrt(np.sum(np.power(np.dot(x_train,w)-y_train,2))/len(x_train))
    if(t%100==0):
        print(f'迭代次数{str(t)}:损失值{str(loss)}')
    gradient=2*np.dot(x_train.transpose(),np.dot(x_train,w)-y_train)
    adagrad+=gradient**2 #**即次方
    w=w-learning_rate*gradient/np.sqrt(adagrad+eps)
np.save('weight.npy',w)

迭代次数0:损失值27.239591682144457
迭代次数100:损失值22.72397574459954
迭代次数200:损失值10.427772652266512
迭代次数300:损失值7.635824772204662
迭代次数400:损失值6.8498786855982985
迭代次数500:损失值6.518567463859376
迭代次数600:损失值6.325643080539458
迭代次数700:损失值6.1943518687643575
迭代次数800:损失值6.099081161908108
迭代次数900:损失值6.027838242824769


#pretesting,对数据进按8：2进行拆分，2成为validation set,用于提前验证private set上结果好坏
x_train=x[:math.floor(len(x)*0.8),:] #math.floor 向下取整
y_train=y[:math.floor(len(y)*0.8),:]
x_validation=x[math.floor(len(x)*0.8):,:]
y_validation=x[math.floor(len(y)*0.8):,:]

w=np.load('weight.npy')
print(len(x_validation))
x_validation=np.concatenate((np.ones([len(x_validation),0]),x_validation),axis=1).astype(float) #ones括号内的0列代表有1列
loss=np.sqrt(np.sum(np.power(np.dot(x_validation,w)-y_validation,2))/len(x_validation))
print(loss)

In [17]:
#用validation来pretesting
w=np.load('weight.npy')
('weight.npy')
x_validation=np.concatenate((np.ones([len(x_validation),0]),x_validation),axis=1).astype(float)
loss=np.sqrt(np.sum(np.power(np.dot(x_validation,w)-y_validation,2))/len(x_validation))

In [18]:
#载入test data并和上面步骤一样进行资料预处理
testdata=pd.read_csv('/home/zjy/ml hw1/test.csv',header=None,encoding = 'big5') #header=None不能漏掉，pd.read_csv会默认把数据第一行内容当标题，导致下面矩阵处理时行数不对应
test_data=testdata.iloc[:,2:] #用ilog有warning
test_data[test_data=='NR']=0
test_data=test_data.to_numpy()
test_x=np.empty([240,18*9],dtype=float)
for i in range(240):
    test_x[i,:]=test_data[18*i:18*(i+1),:].reshape(1,-1) #reshape将其转换为一行（含18*9个元素），与test_x的一行对应
#进行标准化
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j]!=0: #确保分母不为0
            test_x[i][j]=(test_x[i][j]-mean_x[j])/std_x[j]
test_x=np.concatenate((np.ones([240,1]),test_x),axis=1).astype(float)
print(test_x)

[[ 1.         -0.24447681 -0.24545919 ... -0.67065391 -1.04594393
   0.07797893]
 [ 1.         -1.35825331 -1.51789368 ...  0.17279117 -0.10906991
  -0.48454426]
 [ 1.          1.5057434   1.34508393 ... -1.32666675 -1.04594393
  -0.57829812]
 ...
 [ 1.          0.3919669   0.54981237 ...  0.26650729 -0.20275731
   1.20302531]
 [ 1.         -1.8355861  -1.8360023  ... -1.04551839 -1.13963133
  -1.14082131]
 [ 1.         -1.35825331 -1.35883937 ...  2.98427476  3.26367657
   1.76554849]]


/tmp/ipykernel_10102/977769060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_data=='NR']=0
/tmp/ipykernel_10102/977769060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_data=='NR']=0


In [19]:
#对test data进行预测
w=np.load('weight.npy')
ans_y=np.dot(test_x,w)

In [20]:
#将预测结果存入csv文件
import csv
with open('submit.csv',mode='w',newline='')as submit_file:
    csv_writer=csv.writer(submit_file)
    header=['天数','pm2.5预测值']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row=['第'+str(i)+'天',ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)
        

['天数', 'pm2.5预测值']
['第0天', 7.249089768260515]
['第1天', 17.679768702689696]
['第2天', 24.71103166771152]
['第3天', 6.3690988369700605]
['第4天', 28.37337148373311]
['第5天', 21.59201636247953]
['第6天', 22.830842636600664]
['第7天', 31.135374457500454]
['第8天', 18.11812069728249]
['第9天', 57.18105107311826]
['第10天', 14.35679575689403]
['第11天', 9.604122812579146]
['第12天', 60.850885289488346]
['第13天', 51.315642987388664]
['第14天', 20.84716298539006]
['第15天', 11.918588326918645]
['第16天', 31.917031207506483]
['第17天', 67.455594135257]
['第18天', -0.6321568371740547]
['第19天', 16.727993788780754]
['第20天', 42.21696344117322]
['第21天', 71.65542095380837]
['第22天', 9.88201839855455]
['第23天', 18.42548230666614]
['第24天', 15.148573673067979]
['第25天', 38.25844014988789]
['第26天', 14.104449231390834]
['第27天', 75.10179199559501]
['第28天', 7.795149166600199]
['第29天', 54.888967917592346]
['第30天', 22.2796468978286]
['第31天', 8.70958678911193]
['第32天', 4.4155164135662375]
['第33天', 22.802323251380177]
['第34天', 31.204281266013645]